In [ ]:
# Install Essential Libraries
# !pip install ultralytics

In [25]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.models as models
device = "cuda" if torch.cuda.is_available() else "cpu"

In [30]:
class dataloader():
    def __init__(self, mode, transform, bc_df):
        
        self.mode = mode
        self.transform = transform
        
        
        
        train_images, test_images = train_test_split(bc_df, test_size=0.3, random_state=42)
        train_set, val_set = train_test_split(bc_df, test_size=0.2, random_state=42)
        self.train_img, self.train_label = train_set["filepaths"].values, train_set["labels"].values
        self.test_img, self.test_label = test_images["filepaths"].values, test_images["labels"].values
        self.val_img, self.val_label = val_set["filepaths"].values, val_set["labels"].values
        if mode == "train":
            self.img, self.label =  self.train_img, self.train_label
        elif mode == "test":
            self.img, self.label =  self.test_img, self.test_label
        elif mode == "val":
            self.img, self.label =  self.val_img, self.val_label
        
        
        
    def __len__(self):
        return len(self.img)
    
    def __getitem__(self, i):
        
        img_path, label = self.img[i], self.label[i]
        img = Image.open(img_path)
        img = self.transform(img)
        if "NO" in label:
            label = 0
        else:
            label = 1
        
        
        return img, torch.tensor(label)
        
        
    
    

In [31]:
Drowsy='dataset/Driver Drowsiness Dataset (DDD)/Drowsy'
Non_Drowsy="dataset/Driver Drowsiness Dataset (DDD)/Non Drowsy"
filepaths = []
labels = []
dict_list = [Drowsy,Non_Drowsy]
for i, j in enumerate(dict_list):
    flist = os.listdir(j)
    for f in flist:
        fpath = os.path.join(j, f)
        if f[0] != ".":
            filepaths.append(fpath)
            if i == 0:
                labels.append('Drowsy')
            elif i == 1:
                labels.append('Non_Drowsy')
Fseries = pd.Series(filepaths, name="filepaths")
Lseries = pd.Series(labels, name="labels")
bc_data = pd.concat([Fseries, Lseries], axis=1)
bc_df = pd.DataFrame(bc_data)


print(bc_df["labels"].value_counts())


print(bc_df.shape)
bc_df.sample(10)

labels
Drowsy        22348
Non_Drowsy    19445
Name: count, dtype: int64
(41793, 2)


,filepaths,labels
9790,dataset/Driver Drowsiness Dataset (DDD)/Drowsy...,Drowsy
32366,dataset/Driver Drowsiness Dataset (DDD)/Non Dr...,Non_Drowsy
7732,dataset/Driver Drowsiness Dataset (DDD)/Drowsy...,Drowsy
15861,dataset/Driver Drowsiness Dataset (DDD)/Drowsy...,Drowsy
1836,dataset/Driver Drowsiness Dataset (DDD)/Drowsy...,Drowsy
36755,dataset/Driver Drowsiness Dataset (DDD)/Non Dr...,Non_Drowsy
36939,dataset/Driver Drowsiness Dataset (DDD)/Non Dr...,Non_Drowsy
36108,dataset/Driver Drowsiness Dataset (DDD)/Non Dr...,Non_Drowsy
17470,dataset/Driver Drowsiness Dataset (DDD)/Drowsy...,Drowsy
26208,dataset/Driver Drowsiness Dataset (DDD)/Non Dr...,Non_Drowsy


In [32]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((198, 198)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
batch_size = 32
train_dataset = dataloader("train", transform, bc_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

test_dataset = dataloader("test", transform, bc_df)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

val_dataset = dataloader("val", transform, bc_df)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


In [12]:
from ultralytics import YOLO
model = torch.hub.load("ultralytics/yolov5", "custom", path="./yolov5m-cls.pt").to("cuda:0")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple

requirements: AutoUpdate success ✅ 1.8s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-4-6 Python-3.9.19 torch-2.2.2+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)

Fusing layers... 
Model summary: 166 layers, 12947192 parameters, 0 gradients, 31.7 GFLOPs
WARNING ⚠️ YOLOv5 ClassificationModel is not yet AutoShape compatible. You must pass torch tensors in BCHW to this model, i.e. shape(1,3,224,224).


In [13]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
optimizer = optim.Adam(model.parameters(), lr=0.1) 
criterion = nn.CrossEntropyLoss() 


In [36]:
epoch =50
for i in range(epoch):
    losses = 0
    for i, data in tqdm(enumerate(train_loader)):
        labels = torch.zeros((data[1].shape[0], 1000))
        img, label = data[0], data[1]
        img = img.to("cuda:0")
        for i in range(data[1].shape[0]):
            index = data[1][i].item()
            labels[i][index] = 1
        label = labels.to("cuda:0")
        label.requires_grad=True
        result = model(img)
        loss = criterion(result, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses += loss.item()
        
        
    

1045it [11:58,  1.45it/s]


Epoch [25/50], Loss: 9607.8866


1045it [10:59,  1.58it/s]


Epoch [25/50], Loss: 9607.8383


519it [05:03,  1.71it/s]


KeyboardInterrupt: 

In [ ]:
print(dir(train_loader))

In [ ]:
print(dir(train_loader.dataset), train_loader.dataset.train_img)

In [40]:
model.eval()
acc = 0
for i, data in tqdm(enumerate(val_loader)):
    img, label = data[0], data[1]
    img = img.to("cuda:0")
    result = model(img)
    for num in range(len(img)):
        if result[num][0].item() < result[num][1].item():
            output = 1
        else:
            output = 0
    
        if output == label[num].item():
            acc += 1
print(f"{acc} / {len(val_dataset)}")

262it [04:37,  1.06s/it]

750 / 8359
